In [1]:
import math
import statistics
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('../data/Residential-Profiles.csv')
df['Time'] = df['Time'].astype('datetime64')
houses = ["Household " + str(num) for num in range(1, 201)]

#### Case 1: Laplacian noise with peak cutoff 1500 (no carry over)

In [50]:
# Paper reports the average of results from 5 iterations on different peak value [Figure 14 -- blue color bar and blue line]

cutoff = [500, 1000, 1500, 2000, 2500]
print("Case 1: Laplacian noise with peak cutoff 1500 (no carry over)")
for PEAK_VALUE in cutoff:
    print("\nFor peak_value:", PEAK_VALUE)
    summ = []
    pointt = []
    for iteration in range(5):
        epsilon=1
        #PEAK_VALUE=500 
        EX = 0
        total_relative_error = 0

        noisy_energy_list = []
        original_energy_list = []
        time_list = []

        for timestamps in range(4464):
            time = df.at[timestamps,'Time']
            energy = df.at[timestamps,'Household 1']
            original_energy_list.append(energy)

            sensitivity = PEAK_VALUE  # f() --> which will be the next streaming value. 

            #laplace noise
            if energy > PEAK_VALUE:
                energy = PEAK_VALUE

            lap_noise = np.random.laplace(loc=0, scale=sensitivity/epsilon)
            noisy_energy = energy + lap_noise 

            noisy_energy_list.append(noisy_energy)
            time_list.append(time)

        point_re  = (1/4464)*sum(abs(np.array(noisy_energy_list) - np.array(original_energy_list))/ np.array(original_energy_list))
        sum_re = abs(sum(noisy_energy_list) - sum(original_energy_list))*100/sum(original_energy_list)
        
        print('Iteration:', (iteration+1))
       
        print('Relative_Error', sum_re)
        print('MAPE', point_re)
        summ.append(sum_re)
        pointt.append(point_re)
    
    print("\n****************AVG RESULTS of 5 ITERATIONS*************************")
    print("For PEAK VALUE =", PEAK_VALUE) 
    print("\nAVERAGE RELATIVE ERROR:", statistics.mean(summ))
    print("STDEV RELATIVE ERROR:",statistics.pstdev(summ))
    print("AVERAGE MAPE:",statistics.mean(pointt))
    print("STDEV MAPE:",statistics.pstdev(pointt))
    print("********************************************************************\n")



Case 1: Laplacian noise with peak cutoff 1500 (no carry over)

For peak_value: 500
Iteration: 1
Relative_Error 40.483190584548
MAPE 1.1379857161249805
Iteration: 2
Relative_Error 39.09181576099921
MAPE 1.1520816156434428
Iteration: 3
Relative_Error 43.599024624576856
MAPE 1.1847061960890546
Iteration: 4
Relative_Error 39.68330692056836
MAPE 1.164360356501042
Iteration: 5
Relative_Error 39.78466083980494
MAPE 1.115677592022123

****************AVG RESULTS of 5 ITERATIONS*************************
For PEAK VALUE = 500

AVERAGE RELATIVE ERROR: 40.528399746099474
STDEV RELATIVE ERROR: 1.5976159740913534
AVERAGE MAPE: 1.1509622952761285
STDEV MAPE: 0.02337868112162469
********************************************************************


For peak_value: 1000
Iteration: 1
Relative_Error 11.876556314766292
MAPE 2.130470364615601
Iteration: 2
Relative_Error 9.10393453627503
MAPE 2.0625345488366955
Iteration: 3
Relative_Error 6.34098532809666
MAPE 2.1190673586973148
Iteration: 4
Relative_Error 1

### Case 2: Laplacian noise with peak cutoff 1500 (with carry over)

In [49]:
# Paper reports the average of results from 5 iterations on different peak value [Figure 14 -- light orange bar and line]

print("Case 2: Laplacian noise with peak cutoff 1500 (with carry over)")
cutoff = [500, 1000, 1500, 2000, 2500]

for PEAK_VALUE in cutoff:
    print("\nFor peak_value:", PEAK_VALUE)
    summ = []
    pointt = []
    for iteration in range(5):
        epsilon=1
        #PEAK_VALUE=500
        EX = 0
        total_relative_error = 0

        noisy_energy_list = []
        original_energy_list = []
        time_list = []

        for timestamps in range(4464):
            time = df.at[timestamps,'Time']
            energy = df.at[timestamps,'Household 1']
            original_energy_list.append(energy)

            sensitivity = PEAK_VALUE  # f() --> which will be the next streaming value. 

            #laplace noise
            lap_noise = np.random.laplace(loc=0, scale=sensitivity/epsilon)

            if EX == 0:
                if energy > PEAK_VALUE:
                    EX = energy - PEAK_VALUE
                    energy = PEAK_VALUE
                    noisy_energy = energy + lap_noise
                else:
                    noisy_energy = energy + lap_noise
            else:
                if ((energy + EX) <= PEAK_VALUE):
                    noisy_energy = energy + lap_noise  + EX
                    EX = 0
                elif ((energy + EX) > PEAK_VALUE):
                    energy = energy + EX
                    EX = energy - PEAK_VALUE
                    energy = PEAK_VALUE
                    noisy_energy = energy + lap_noise 





            noisy_energy_list.append(noisy_energy)
            time_list.append(time)

        point_re  = (1/4464)*sum(abs(np.array(noisy_energy_list) - np.array(original_energy_list))/ np.array(original_energy_list))
        sum_re = abs(sum(noisy_energy_list) - sum(original_energy_list))*100/sum(original_energy_list)

        print('Iteration:', (iteration+1))
        print('Relative_Error', sum_re)
        print('MAPE', point_re)
        summ.append(sum_re)
        pointt.append(point_re)
    
    print("\n****************AVG RESULTS of 5 ITERATIONS*************************")
    print("For PEAK VALUE =", PEAK_VALUE) 
    print("\nAVERAGE RELATIVE ERROR:", statistics.mean(summ))
    print("STDEV RELATIVE ERROR:",statistics.pstdev(summ))
    print("AVERAGE MAPE:",statistics.mean(pointt))
    print("STDEV MAPE:",statistics.pstdev(pointt))
    print("********************************************************************\n")

    



Case 2: Laplacian noise with peak cutoff 1500 (with carry over)

For peak_value: 500
Iteration: 1
Relative_Error 24.995341443343186
MAPE 1.2167759784144587
Iteration: 2
Relative_Error 27.412659869805218
MAPE 1.203650733858825
Iteration: 3
Relative_Error 27.893522364722152
MAPE 1.202860980518522
Iteration: 4
Relative_Error 27.69166288407948
MAPE 1.2217456099572006
Iteration: 5
Relative_Error 29.176997401089785
MAPE 1.1925707141422246

****************AVG RESULTS of 5 ITERATIONS*************************
For PEAK VALUE = 500

AVERAGE RELATIVE ERROR: 27.434036792607966
STDEV RELATIVE ERROR: 1.3611099776330704
AVERAGE MAPE: 1.207520803378246
STDEV MAPE: 0.010470911540966882
********************************************************************


For peak_value: 1000
Iteration: 1
Relative_Error 0.4676927874675596
MAPE 2.1488909239796024
Iteration: 2
Relative_Error 1.5137031585868688
MAPE 2.224927891630433
Iteration: 3
Relative_Error 0.8936597041270835
MAPE 2.186230222154651
Iteration: 4
Relati

### Case 3: Different time different sensitivity

In [47]:
# Paper reports the average of results from 5 iterations on different peak value [Figure 15 -- light orange bar and line]
# cutoff = [500, 1000, 1500, 2000, 2500]
from datetime import time
import statistics

# change day and night Peak energy value here
DAY_PEAK = 1000
NIGHT_PEAK = 250

summ = []
pointt = []
for trial in range(5):
    epsilon=1


    EX = 0
    total_relative_error = 0

    noisy_energy_list = []
    original_energy_list = []
    time_list = []

    time_morning = time(5, 59, 59, 999999)  # 6am
    time_night = time(23, 59, 59, 999999) # 12am

    for timestamps in range(4464):
        t = df.at[timestamps,'Time']
        energy = df.at[timestamps,'Household 1']
        original_energy_list.append(energy)

        if ((t.time()>= time_morning)  & (t.time()< time_night)):
            sensitivity = DAY_PEAK          # PEAK energy value at day
        else:
            sensitivity = NIGHT_PEAK           # PEAK energy value at night
            


        #laplace noise
        if energy > sensitivity:
            energy = sensitivity

        lap_noise = np.random.laplace(loc=0, scale=sensitivity/epsilon)
        noisy_energy = energy + lap_noise 

        noisy_energy_list.append(noisy_energy)
        time_list.append(t)

    point_re  = (1/4464)*sum(abs(np.array(noisy_energy_list) - np.array(original_energy_list))/ np.array(original_energy_list))
    sum_re = abs(sum(noisy_energy_list) - sum(original_energy_list))*100/sum(original_energy_list)

    print('Iteration:', (trial+1))
    print('Relative_Error', sum_re)
    print('MAPE', point_re)
        
    summ.append(sum_re)
    pointt.append(point_re)
    

print("\n****************AVG RESULTS of 5 ITERATIONS*************************")
print("For DAY PEAK (D) =", DAY_PEAK , " & NIGNT PEAK (N) =", NIGHT_PEAK) 
print("\nAVERAGE RELATIVE ERROR:", statistics.mean(summ))
print("STDEV RELATIVE ERROR:",statistics.pstdev(summ))
print("AVERAGE MAPE:",statistics.mean(pointt))
print("STDEV MAPE:",statistics.pstdev(pointt))
print("********************************************************************\n")



Iteration: 1
Relative_Error 16.18131555492716
MAPE 1.644039763341966
Iteration: 2
Relative_Error 22.087643243219887
MAPE 1.586969592475568
Iteration: 3
Relative_Error 21.005399821152196
MAPE 1.6819251690693262
Iteration: 4
Relative_Error 23.864980341521008
MAPE 1.6540217641252666
Iteration: 5
Relative_Error 15.266322146259633
MAPE 1.6465737928453472

****************AVG RESULTS of 5 ITERATIONS*************************
For DAY PEAK (D) = 1000  & NIGNT PEAK (N) = 250

AVERAGE RELATIVE ERROR: 19.681132221415975
STDEV RELATIVE ERROR: 3.3701284772131785
AVERAGE MAPE: 1.6427060163714948
STDEV MAPE: 0.030949880807855056
********************************************************************



# Case 4: Case 3 with carry on the noises

In [48]:
# cutoff = [500, 1000, 1500, 2000, 2500]
from datetime import time
import statistics

# change day and night Peak energy value here
DAY_PEAK = 1000
NIGHT_PEAK = 250

summ = []
pointt = []
for trial in range(5):
    epsilon=1

    EX = 0
    total_relative_error = 0

    noisy_energy_list = []
    original_energy_list = []
    time_list = []

    time_morning = time(5, 59, 59, 999999)  # 6am
    time_night = time(23, 59, 59, 999999) # 12am

    for timestamps in range(4464):
        t = df.at[timestamps,'Time']
        energy = df.at[timestamps,'Household 1']
        original_energy_list.append(energy)

        if ((t.time()>= time_morning)  & (t.time()< time_night)):
            sensitivity = DAY_PEAK
        else:
            sensitivity = NIGHT_PEAK

        PEAK_VALUE = sensitivity
        #laplace noise
        lap_noise = np.random.laplace(loc=0, scale=sensitivity/epsilon)

        if EX == 0:
            if energy > PEAK_VALUE:
                EX = energy - PEAK_VALUE
                energy = PEAK_VALUE
                noisy_energy = energy + lap_noise
            else:
                noisy_energy = energy + lap_noise
        else:
            if ((energy + EX) <= PEAK_VALUE):
                noisy_energy = energy + lap_noise  + EX
                EX = 0
            elif ((energy + EX) > PEAK_VALUE):
                energy = energy + EX
                EX = energy - PEAK_VALUE
                energy = PEAK_VALUE
                noisy_energy = energy + lap_noise  


        noisy_energy_list.append(noisy_energy)
        time_list.append(t)

    point_re  = (1/4464)*sum(abs(np.array(noisy_energy_list) - np.array(original_energy_list))/ np.array(original_energy_list))
    sum_re = abs(sum(noisy_energy_list) - sum(original_energy_list))*100/sum(original_energy_list)

    print('Iteration:', (trial+1))
    print('Relative_Error', sum_re)
    print('MAPE', point_re)
    
    summ.append(sum_re)
    pointt.append(point_re)
    

print("\n****************AVG RESULTS of 5 ITERATIONS*************************")
print("For DAY PEAK (D) =", DAY_PEAK , " & NIGNT PEAK (N) =", NIGHT_PEAK) 
print("\nAVERAGE RELATIVE ERROR:", statistics.mean(summ))
print("STDEV RELATIVE ERROR:",statistics.pstdev(summ))
print("AVERAGE MAPE:",statistics.mean(pointt))
print("STDEV MAPE:",statistics.pstdev(pointt))
print("********************************************************************\n")



Iteration: 1
Relative_Error 4.286156348174582
MAPE 1.68709304698934
Iteration: 2
Relative_Error 7.674673854973373
MAPE 1.7048249043837196
Iteration: 3
Relative_Error 2.951163975345177
MAPE 1.7072535963799262
Iteration: 4
Relative_Error 3.6612248520790143
MAPE 1.7162611248235766
Iteration: 5
Relative_Error 3.108479801309506
MAPE 1.7040275843971493

****************AVG RESULTS of 5 ITERATIONS*************************
For DAY PEAK (D) = 1000  & NIGNT PEAK (N) = 250

AVERAGE RELATIVE ERROR: 4.33633976637633
STDEV RELATIVE ERROR: 1.7337315291550084
AVERAGE MAPE: 1.7038920513947424
STDEV MAPE: 0.009459271590157714
********************************************************************



# Different noise combination

In [52]:
# Paper reports the average of results from 5 iterations on different peak value [Figure 16]
# Showing the results for Laplace + Exponential combination
# to see the Gaussian and Gamma combination -- Please uncomment the 5 lines following the comment '#Gaussian + Gamma'

print("Case 5: Different noise combination")
cutoff = [500, 1000, 1500, 2000, 2500]

for PEAK_VALUE in cutoff:
    print("\nFor peak_value:", PEAK_VALUE)
    summ = []
    pointt = []
    for iteration in range(5):
        epsilon=0.5   # noise combination will make the epsilon=1
        #PEAK_VALUE=2500 
        delta = 10e-1
        total_relative_error = 0

        noisy_energy_list = []
        original_energy_list = []
        time_list = []

        for timestamps in range(4464):
            time = df.at[timestamps,'Time']
            energy = df.at[timestamps,'Household 1']
            original_energy_list.append(energy)

            sensitivity = PEAK_VALUE  # f() --> which will be the next streaming value. 

            #laplace noise
            if energy > PEAK_VALUE:
                energy = PEAK_VALUE
            
            # laplace + Exponential
            noise1 = np.random.laplace(loc=0, scale=sensitivity/epsilon)    # laplace
            noise2 = np.random.exponential(scale=sensitivity/epsilon)       # exponential

            #Gaussian + Gamma   
            #sigma = np.sqrt(2*np.log(1.25/delta)) * sensitivity / epsilon
            #noise1 = np.random.normal(loc=0, scale = sigma)                         # Gaussian
            #gamma1 = np.random.gamma(shape=(1/4464), scale = sensitivity/epsilon)
            #gamma2 = np.random.gamma(shape=(1/4464), scale = sensitivity/epsilon)
            #noise2 = (gamma1-gamma2)                                                # Gamma

            total_noise = (noise1 + noise2)/2

            noisy_energy = (energy + total_noise)

            noisy_energy_list.append(noisy_energy)
            time_list.append(time)


        point_re  = (1/4464)*sum(abs(np.array(noisy_energy_list) - np.array(original_energy_list))/ np.array(original_energy_list))
        sum_re = abs(sum(noisy_energy_list) - sum(original_energy_list))*100/sum(original_energy_list)

        print('Iteration:', (iteration+1))
        print('Relative_Error', sum_re)
        print('MAPE', point_re)
        summ.append(sum_re)
        pointt.append(point_re)
        
    print("\n****************AVG RESULTS of 5 ITERATIONS*************************")
    print("For PEAK VALUE =", PEAK_VALUE , "& Combination of LAPLACE and EXPONENTIAL") 
    print("\nAVERAGE RELATIVE ERROR:", statistics.mean(summ))
    print("STDEV RELATIVE ERROR:",statistics.pstdev(summ))
    print("AVERAGE MAPE:",statistics.mean(pointt))
    print("STDEV MAPE:",statistics.pstdev(pointt))
    print("********************************************************************\n")




Case 5: Different noise combination

For peak_value: 500
Iteration: 1
Relative_Error 31.089348249566154
MAPE 1.5644233671811953
Iteration: 2
Relative_Error 31.780528909771633
MAPE 1.56489945486058
Iteration: 3
Relative_Error 34.430997610317945
MAPE 1.5562662922239685
Iteration: 4
Relative_Error 27.44459792766495
MAPE 1.5415021114176017
Iteration: 5
Relative_Error 33.167737211238595
MAPE 1.5346776834952232

****************AVG RESULTS of 5 ITERATIONS*************************
For PEAK VALUE = 500 & Combination of LAPLACE and EXPONENTIAL

AVERAGE RELATIVE ERROR: 31.582641981711856
STDEV RELATIVE ERROR: 2.36776532660585
AVERAGE MAPE: 1.5523537818357138
STDEV MAPE: 0.01223584686895796
********************************************************************


For peak_value: 1000
Iteration: 1
Relative_Error 136.52630589961285
MAPE 3.167640671386816
Iteration: 2
Relative_Error 130.89290898702242
MAPE 3.1824374714750636
Iteration: 3
Relative_Error 131.6947837874143
MAPE 3.160820132573638
Iteration

# Add smoothing

In [30]:
def smoothing(my_list):
    length = len(my_list)
    avg_list = [0]*length
    median_list = [0]*length
    for i in range(1, len(my_list)-1):
        avg_list[i] = (my_list[i-1] + my_list[i] + my_list[i+1])/3
        median_list[i] = sorted([my_list[i-1], my_list[i], my_list[i+1]])[1]
    
    avg_list[0] = my_list[0]
    avg_list[length-1] = my_list[length-1]
    median_list[length-1] = my_list[length-1]
    median_list[0] = my_list[0]
    return avg_list, median_list

# avg, med = smoothing([1,5,3,9,5])
# print(avg)
# print(med)

In [57]:
# Paper reports the average of results from 5 iterations on different peak value [Figure 17]
print("Smoothing techniques")
cutoff = [500, 1000, 1500, 2000, 2500, 3000]
for PEAK_VALUE in cutoff:
    print("\nFor peak_value:", PEAK_VALUE)
    summ_noisy = []
    pointt_noisy = []
    summ_avg_smoothing = []
    pointt_avg_smoothing = []
    summ_median_smoothing = []
    pointt_median_smoothing = []
    for abc in range(5):
        epsilon=1
        #PEAK_VALUE=1000
        EX = 0
        total_relative_error = 0

        noisy_energy_list = []
        original_energy_list = []
        time_list = []

        for timestamps in range(4464):
            time = df.at[timestamps,'Time']
            energy = df.at[timestamps,'Household 1']
            original_energy_list.append(energy)

            sensitivity = PEAK_VALUE  # f() --> which will be the next streaming value. 

            #laplace noise
            if energy > PEAK_VALUE:
                energy = PEAK_VALUE

            lap_noise = np.random.laplace(loc=0, scale=sensitivity/epsilon)
            noisy_energy = energy + lap_noise 

            noisy_energy_list.append(noisy_energy)
            time_list.append(time)

        # calculating the average and median of generated noises 
        avg_list, median_list = smoothing(noisy_energy_list)
        #print(sum(original_energy_list))
        #print('noisy: ', sum(noisy_energy_list))
        #print('median smoothing: ', sum(median_list))
        #print('average smoothing: ', sum(avg_list))

        point_re_noisy  = (1/4464)*sum(abs(np.array(noisy_energy_list) - np.array(original_energy_list))/ np.array(original_energy_list))
        point_re_avg  = (1/4464)*sum(abs(np.array(avg_list) - np.array(original_energy_list))/ np.array(original_energy_list))
        point_re_med  = (1/4464)*sum(abs(np.array(median_list) - np.array(original_energy_list))/ np.array(original_energy_list))
        sum_re_noisy = abs(sum(noisy_energy_list) - sum(original_energy_list))*100/sum(original_energy_list)

        sum_re_avg = abs(sum(avg_list) - sum(original_energy_list))*100/sum(original_energy_list)
        sum_re_med = abs(sum(median_list) - sum(original_energy_list))*100/sum(original_energy_list)
        
        print('Iteration:', (abc+1))
        print('Relative Error (Noisy):', sum_re_noisy)
        print('Relative Error (Average Smoothing):', sum_re_avg)
        print('Relative Error (Median Smoothing):', sum_re_med)
        print('MAPE (Noisy):', point_re_noisy)
        print('MAPE (Average Smoothing):', point_re_avg)
        print('MAPE (Median Smoothing):', point_re_med)
        
        summ_noisy.append(sum_re_noisy)
        pointt_noisy.append(point_re_noisy)
        
        summ_avg_smoothing.append(sum_re_avg)
        pointt_avg_smoothing.append(point_re_avg)
        
        summ_median_smoothing.append(sum_re_med)
        pointt_median_smoothing.append(point_re_med)
        
        
        
    print("\n****************AVG RESULTS of 5 ITERATIONS*************************")
    print("For PEAK VALUE =", PEAK_VALUE ) 
    print("\nNOISY:")
    print("AVERAGE RELATIVE ERROR:", statistics.mean(summ_noisy))
    print("STDEV RELATIVE ERROR:",statistics.pstdev(summ_noisy))
    print("AVERAGE MAPE:",statistics.mean(pointt_noisy))
    print("STDEV MAPE:",statistics.pstdev(pointt_noisy))
    print("\nAVERAGE SMOOTHING:")
    print("AVERAGE RELATIVE ERROR:", statistics.mean(summ_avg_smoothing))
    print("STDEV RELATIVE ERROR:",statistics.pstdev(summ_avg_smoothing))
    print("AVERAGE MAPE:",statistics.mean(pointt_avg_smoothing))
    print("STDEV MAPE:",statistics.pstdev(pointt_avg_smoothing))
    print("\nMEDIAN SMOOTHING:")
    print("AVERAGE RELATIVE ERROR:", statistics.mean(summ_median_smoothing))
    print("STDEV RELATIVE ERROR:",statistics.pstdev(summ_median_smoothing))
    print("AVERAGE MAPE:",statistics.mean(pointt_median_smoothing))
    print("STDEV MAPE:",statistics.pstdev(pointt_median_smoothing))
    print("********************************************************************\n")




Smoothing techniques

For peak_value: 500
Iteration: 1
Relative Error (Noisy): 39.95879534289871
Relative Error (Average Smoothing): 39.95216074785062
Relative Error (Median Smoothing): 41.60972540354034
MAPE (Noisy): 1.1686720276809934
MAPE (Average Smoothing): 0.8052761121142846
MAPE (Median Smoothing): 0.7705666279144544
Iteration: 2
Relative Error (Noisy): 37.4258993199965
Relative Error (Average Smoothing): 37.4311948819753
Relative Error (Median Smoothing): 39.30872921073128
MAPE (Noisy): 1.1465493333854944
MAPE (Average Smoothing): 0.7887647238748321
MAPE (Median Smoothing): 0.7712625855893784
Iteration: 3
Relative Error (Noisy): 38.98130658388939
Relative Error (Average Smoothing): 38.979601063055654
Relative Error (Median Smoothing): 40.130473243146575
MAPE (Noisy): 1.15875939320794
MAPE (Average Smoothing): 0.8083296597977124
MAPE (Median Smoothing): 0.7804026699266932
Iteration: 4
Relative Error (Noisy): 38.96594548799167
Relative Error (Average Smoothing): 38.93014636726659

Iteration: 1
Relative Error (Noisy): 0.6348788139239986
Relative Error (Average Smoothing): 0.6368703019242521
Relative Error (Median Smoothing): 2.70084056447205
MAPE (Noisy): 5.3803565813123635
MAPE (Average Smoothing): 3.350626710798833
MAPE (Median Smoothing): 3.1525466718135253
Iteration: 2
Relative Error (Noisy): 4.305839912977918
Relative Error (Average Smoothing): 4.311782693958346
Relative Error (Median Smoothing): 6.819762896519828
MAPE (Noisy): 5.2794839515709455
MAPE (Average Smoothing): 3.3000895503360796
MAPE (Median Smoothing): 3.1603914769418453
Iteration: 3
Relative Error (Noisy): 0.27637207041531126
Relative Error (Average Smoothing): 0.29803028917509733
Relative Error (Median Smoothing): 4.869513797163093
MAPE (Noisy): 5.2144114137012405
MAPE (Average Smoothing): 3.3076110981756925
MAPE (Median Smoothing): 3.0916898991129167
Iteration: 4
Relative Error (Noisy): 15.62148975242183
Relative Error (Average Smoothing): 15.579632011392174
Relative Error (Median Smoothing):